Hier ist ein umfassendes Testing-Konzept basierend auf der Testpyramide, speziell zugeschnitten auf deine Anwendung (FastAPI + NiceGUI + Azure SQL).

Das Konzept gliedert sich in drei Ebenen: Unit Tests (Basis), Integration Tests (Mitte) und E2E UI Tests mit Selenium (Spitze).

1. Das Konzept: Die Testpyramide
Ebene 1: Unit Tests (Basis - ca. 70%)
Hier testen wir isolierte Logik ohne Datenbank- oder API-Verbindungen.

Ziel: Validierung von Hilfsfunktionen und Business-Logik.
Kandidaten in deinem Code:
db.py: _hash_password (Kryptographie-Logik) -> Corinne.
receipt_analysis.py: _parse_response (Parsing-Logik) -> Roy.
Tool: pytest.


Ebene 2: Integration Tests (Mitte - ca. 20%)
Hier testen wir das Zusammenspiel von Komponenten (z.B. API-Endpunkt -> Mock-DB).

Ziel: Sicherstellen, dass die API-Endpunkte korrekt antworten und Datenbank-Queries syntaktisch korrekt sind (oft mit einer Test-Datenbank oder Mocking).
Kandidaten:
main.py: Testen der Endpunkte /api/upload -> Bellinda 
                            und /api/receipts/ -> Giulio
Tool: pytest, FastAPI TestClient.


Ebene 3: End-to-End (E2E) UI Tests (Spitze - ca. 10%) 
Hier testen wir den gesamten Prozess aus Nutzersicht im Browser.

Ziel: Sicherstellen, dass der Login, der Upload und die Anzeige funktionieren.
Szenario: User öffnet Seite -> Loggt sich ein -> Lädt Beleg hoch -> Sieht Ergebnis. -> Markus
Tool: Selenium WebDriver, pytest.
2. Implementierung
Hier sind die konkreten Implementierungen für die neuen Test-Dateien. Ich gehe davon aus, dass du einen Ordner tests im Projektverzeichnis erstellst.

Vorbereitung (Requirements)
Du benötigst folgende Pakete:

bash
pip install pytest httpx selenium webdriver-manager
A. Unit Tests (Logik prüfen)
Wir testen die Parsing-Logik in receipt_analysis.py, da diese fehleranfällig ist, wenn die KI seltsames JSON liefert.

python
 Show full code block 
# tests/unit/test_analysis_logic.py
import pytest
from decimal import Decimal
from datetime import date
from app.receipt_analysis import ReceiptAnalyzer

def test_parse_response_valid_json():
    """Prüft, ob valides JSON korrekt geparst wird."""
    json_text = '{"total_amount": 12.50, "currency": "CHF"}'
    result = ReceiptAnalyzer._parse_response(json_text)
    assert result["total_amount"] == 12.50
    assert result["currency"] == "CHF"

def test_parse_response_with_markdown():
    """Prüft, ob JSON innerhalb von Markdown-Blöcken erkannt wird (typisch für LLMs)."""
    json_text = '```json\n{"total_amount": 100}\n```'
    result = ReceiptAnalyzer._parse_response(json_text)
    assert result["total_amount"] == 100

def test_to_decimal_conversion():
    """Prüft die sichere Umwandlung in Decimal."""
    assert ReceiptAnalyzer._to_decimal(10.5) == Decimal("10.5")
    assert ReceiptAnalyzer._to_decimal("15.00") == Decimal("15.00")
    assert ReceiptAnalyzer._to_decimal(None) is None
    assert ReceiptAnalyzer._to_decimal("invalid") is None

def test_safe_str_cleaning():
    """Prüft, ob Strings bereinigt werden."""
    assert ReceiptAnalyzer._safe_str("  Hallo  ") == "Hallo"
    assert ReceiptAnalyzer._safe_str(None) is None
B. Integration Tests (API prüfen)
Wir nutzen den TestClient von FastAPI, um die API in main.py zu testen, ohne den Server wirklich starten zu müssen. Wir mocken die Datenbankaufrufe, um keine echte DB zu benötigen.

python
 Show full code block 
# tests/integration/test_api.py
import pytest
from fastapi.testclient import TestClient
from unittest.mock import patch, AsyncMock
from app.main import app

client = TestClient(app)

@patch("app.main.process_receipt_upload")
@patch("app.main.analyze_receipt")
def test_api_upload_endpoint(mock_analyze, mock_process):
    """
    Testet den Upload-Endpunkt.
    Wir simulieren, dass process_receipt_upload und analyze_receipt erfolgreich sind.
    """
    # Setup Mocks
    mock_process.return_value = {"receipt_id": 123, "status": "uploaded"}
    mock_analyze.return_value = {"status": "processed", "transaction": {"amount": 10}}

    # Datei simulieren
    files = {"file": ("test_receipt.jpg", b"fake_image_content", "image/jpeg")}
    data = {"user_id": "1"}

    response = client.post("/api/upload", files=files, data=data)

    assert response.status_code == 200
    json_resp = response.json()
    assert json_resp["receipt_id"] == 123
    assert json_resp["analysis"]["status"] == "processed"
C. E2E Tests mit Selenium (UI prüfen)
Dies ist der komplexeste Teil. Da NiceGUI asynchron läuft, müssen wir für den Test den Uvicorn-Server in einem separaten Thread starten und dann mit Selenium darauf zugreifen.

Hinweis: Da ich den Code der Login-Seite (app/ui_pages/login_page.py) nicht sehe, gehe ich davon aus, dass es Input-Felder für E-Mail und Passwort gibt. Selenium sucht diese hier generisch über den input-Tag.

python
 Show full code block 
# tests/e2e/test_selenium_flow.py
import pytest
import threading
import time
import uvicorn
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from app.main import app

# Server-Konfiguration für den Test
PORT = 8001
HOST = "127.0.0.1"
BASE_URL = f"http://{HOST}:{PORT}"

class UvicornThread(threading.Thread):
    """Startet den FastAPI/NiceGUI Server in einem Hintergrund-Thread."""
    def __init__(self):
        super().__init__()
        self.server = uvicorn.Server(config=uvicorn.Config(
            app, host=HOST, port=PORT, log_level="critical"
        ))
        self.daemon = True

    def run(self):
        self.server.run()

    def stop(self):
        self.server.should_exit = True

@pytest.fixture(scope="module")
def test_server():
    """Fixture, die den Server vor den Tests startet und danach stoppt."""
    server_thread = UvicornThread()
    server_thread.start()
    # Warten, bis der Server hochgefahren ist
    time.sleep(2) 
    yield
    server_thread.stop()

@pytest.fixture(scope="module")
def driver():
    """Fixture für den Selenium WebDriver (Chrome)."""
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Einkommentieren für Headless-Modus (ohne UI)
    
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.implicitly_wait(5) # Warte bis zu 5 Sek auf Elemente
    yield driver
    driver.quit()

def test_login_and_dashboard_access(test_server, driver):
    """
    E2E Test:
    1. Öffnet die Startseite.
    2. Prüft, ob wir auf Login umgeleitet werden (oder Login-Elemente da sind).
    3. Führt einen Login durch (Achtung: Benötigt User in der DB oder Mock).
    """
    driver.get(BASE_URL)
    
    # Annahme: NiceGUI rendert Inputs als <input>. 
    # Wir suchen nach E-Mail und Passwort Feldern.
    # Da NiceGUI oft Quasar nutzt, sind die Inputs oft in verschachtelten Divs.
    # Wir suchen generisch nach Inputs.
    
    # HINWEIS: Für stabile Tests sollten in den .py Dateien der UI 
    # den Elementen props='data-testid=login-email' gegeben werden.
    # Hier versuchen wir es generisch:
    
    time.sleep(1) # Kurz warten bis JS geladen ist
    
    # Prüfen, ob "Smart Expense Tracker" im Titel oder auf der Seite steht
    assert "Smart Expense Tracker" in driver.title or "Smart Expense Tracker" in driver.page_source

    # Beispielhafter Login-Flow (Pseudo-Code, da UI-Code fehlt):
    # inputs = driver.find_elements(By.TAG_NAME, "input")
    # if len(inputs) >= 2:
    #     email_input = inputs[0]
    #     password_input = inputs[1]
    #
    #     email_input.send_keys("test@example.com")
    #     password_input.send_keys("password123")
    #
    #     # Login Button suchen (z.B. über Text)
    #     buttons = driver.find_elements(By.TAG_NAME, "button")
    #     for btn in buttons:
    #         if "Login" in btn.text or "Anmelden" in btn.text:
    #             btn.click()
    #             break
    #     
    #     time.sleep(2)
    #     # Prüfen ob Dashboard geladen (z.B. URL Check)
    #     # assert "dashboard" in driver.current_url
3. Anpassungsvorschlag für bessere Testbarkeit
Damit Selenium Elemente zuverlässig findet, ist es "Best Practice", in NiceGUI eindeutige Test-IDs zu vergeben.

Ich schlage vor, in deinen UI-Dateien (die hier nicht im Kontext waren, aber vermutlich existieren) Attribute hinzuzufügen. Da ich main.py habe, zeige ich hier exemplarisch, wie man das Prinzip anwendet, falls dort UI-Elemente wären (aktuell sind dort nur Routen).

Da keine direkten UI-Elemente in den bereitgestellten Dateien definiert sind (diese werden importiert), hier ein Beispiel, wie du deine login_page.py (hypothetisch) anpassen solltest, um das Testing-Konzept zu unterstützen:

python
# Beispiel (nicht ausführen, nur als Referenz für deine UI-Dateien):
# ui.input('Email').props('data-testid=email-input')
# ui.button('Login', on_click=login).props('data-testid=login-btn')
Dann könntest du im Selenium Test exakt darauf zugreifen:

python
driver.find_element(By.CSS_SELECTOR, "[data-testid='email-input']")
Zusammenfassung
Unit Tests in tests/unit/ sichern die KI-Antwort-Verarbeitung ab.
Integration Tests in tests/integration/ prüfen die FastAPI-Endpunkte.
Selenium Tests in tests/e2e/ fahren die App hoch und klicken sich durch.
Dieses Konzept deckt die Pyramide ab und erfüllt deine Anforderung nach Selenium und UI Tests.

